# P controller on helicopter motion from hover

In this program a simulation example is given of a helicopter in hovering flight when the pilot gives a control input in longitudinal stick and the helicopter react to this input. The program is built in  six steps:

1.	INPUT DATA : Initialization of all necessary helicopter data to be used in the helicopter 3 degrees of freedom longitudinal model;
2.	INITIALIZATION: Initialization of all the necessary input variables of
3.	STARTING THE SIMULATION: In this step the pilot input is given and the loop of the simulation starts. 
4. STARTING THE CONTROLLER:
4.	CALCULATION PARAMETERS: Calculation of different necessary parameters to be used in helicopter equations of motion such as angle of attack of the control plane, rotor flapping angle, rotor thrust coefficient (both in Glauert and Blade element theory).
5.	EQUATIONS OF MOTION: Define the 3 degree of freedom equations of motion
6.	NUMERICAL INTEGRATION: The equations of motion are numerically integrated in order to represent the helicopter motion
7.	PLOT RESULTS: Different variables can be plotted 


## 1. Input data

The following helicopter data are necessary for the program: 

| airplane parameters | control variables | state variables |
|---------------------|-------------------|-----------------|
| $M, I_y, C_D$      | $\theta_0, \theta_e$ | $u, w, q, \theta$ |



In [ ]:
## Input dataimport numpy as np

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np

# INITIAL DATA HELICOPTER
g = 9.81                        # (kg/m^2) Gravitational acceleration
cla = 5.7                       # (rad^-1) Profile NACA 0012 Lift Coefficient slope
volh = .075                     # blade solidity
lok = 6                         # Lock number
cds = 1.5                       # Drag coefficinet fo the fuselage
mass = 2200                     # Helicopter mass
rho = 1.225                     # (kg/m^3) Air density
vtip = 200                      # (m/sec) Blade tip speed
diam = 2 * 7.32                 # (m) Rotor diameter
iy = 10615                      # kg m^2) Helicopter moment of inertia in the pitch axis
mast = 1                        # %(m) Vertical distance between rotor CG and rotor hub
omega = vtip / (diam / 2)       # Rotor tip speed
area = np.pi / 4 * diam ** 2    # Rotor area
tau = .1                        # (sec) time constant in dynamics inflow!!!
collect = [6*np.pi/180]         # (rad)  Collective pitch anlg3
longit = [0*np.pi/180]          # (rad) Longitudinal cyclic pitch angle

## 2. INITIALIZATION

Next, the initialization of the state variables and the pilot control inputs takes place. One can see that we set as initial condition the hovering flight as u=w=0 m/sec.


In [ ]:
# INITIAL VALUES SIMULATION
t0 = 0                                      #(sec) Setting initial time 
u0 = 0                                      #(m/sec) Setting initial helicopter airspeed component along body x-axis 
w0 = 0                                      #(m/sec) Setting initial helicopter airspeed component along body z-axis
q0 = 0                                      #(rad/sec) Setting initial helicopter pitch rate 
pitch0 = 0 * np.pi / 180                    #(rad) Setting initial helicopter pitch angle
x0 = 0                                      #(m) Setting initial helicopter longitudinal position 
labi0 = np.sqrt(mass*g/(area*2*rho))/vtip   # Initilization nondimentional inflow

## 3.SIMULATION

We have now all the variables defined to start the simulation and apply pilot control input. As pilot input we will consider the following: Between t = 0.5 sec and t = 1 sec the cyclic stick is pushed forward, such that a cyclic pitch of $\theta_c=0^{\circ}$ (deg) results. Outside this time frame the stick is held in the neutral position (cyclic pitch $\theta_c=0^{\circ}$).

## 4. Starting the controller

When starting the simulation we considered that the pilot is giving an input as followings: Between t = 0.5 sec and t = 1 sec the cyclic stick is pushed forward, such that a cyclic pitch of $\theta_c = 1°$ results. Outside this time frame the stick is held in the neutral position (cyclic pitch $\theta_c = 0°$). Now, from the 15th second of the simulation we will consider a P controller for the longitudinal cyclic pitch that becomes active and tries to stabilize the motion of the helicopter. For the proportional P controller the following scheme can be applied. 

<br>
<p align="center"  id="figure1">
<img src="https://github.com/MDPAV/HELICOR/blob/main/figures/p_controller.png?raw=true" width="500" /> 
 <br>
<em>Figure 1: Free body diagram of a pitching helicopter</em>
</p>
<br>

Where $\theta_f$  is the fuselage pitch angle, $\theta_{fwens}$  is the wishes fuselage pitch angle of the helicopter that the pilot needs to maintain (in this case the goal is to hover so $\theta_{fwens} = 0  ) and K is the gain of the P controller. This results in the following law for the P-controller:

$$
\theta_c = K \cdot (\theta_f - \theta_{fwens})
$$
 
This law is applied from the 15th second of the simulation for this particular simulation case.


## 5.Calculation of parameters
The following parameters need to be calculated: total helicopter velocity  $V$ , angle of attack of the control plane  $\alpha_c$ , advance ratio $\mu$ , perpendicular component of the total velocity on the control plane  $\lambda_c$ :

$$
\begin{align*}
V &= \sqrt{u^2 + w^2} \\
\alpha_c &= \theta_c - \arctan\left(\frac{w}{u}\right) \\
\mu &= \frac{V}{\Omega R} \cdot \cos(\alpha_c) \\
\lambda_c &= \frac{V}{\Omega R} \cdot \sin(\alpha_c)
\end{align*}
$$

> **Attention**: when calculating the angle of attack, please take into account that, during the simulation the situation can arise of flying backwards. Take care and compute in these cases the proper quadrant during the simulation. In order to do this one should take care that if  $u < 0$  then  $\alpha_c + \pi$.


Also, in this part of the simulation one needs to calculate the rotor longitudinal flapping angle $a_1$ and rotor thrust $C_T$ coefficient both in blade element theory $C_{T_{BEM}}$ and Glauert theory $C_{T_{Glau}}$:

$$
\begin{align*}
a_1 &= \frac{\frac{8}{3} \mu \theta_0 - 2\mu(\lambda_c + \lambda_i) - \frac{16}{\gamma} \frac{q}{\Omega}}{1 - \frac{1}{2} \mu^2} \\
C_{T_{BEM}} &= \frac{1}{4} c_{l\alpha} \sigma \left[ \frac{2}{3} \left(1 + \frac{3}{2} \mu^2 \right) - (\lambda_c + \lambda_i) \right] \\
C_{T_{Glau}} &= 2 \lambda_i \sqrt{ \left( \frac{V}{\Omega R} \cos(\alpha_c - a_1) \right)^2 + \left( \frac{V}{\Omega R} \sin(\alpha_c - a_1) + \lambda_i \right)^2 }
\end{align*}
$$





In [ ]:

t = [t0]
u = [u0]
w = [w0]
q = [q0]
pitch = [pitch0]
x = [x0]
labi = [labi0]
z = [0]

# Integration
aantal = 800
teind = 80
stap = (teind - t0) / aantal

for i in range(aantal):
    if t[i] >= 0.5 and t[i] <= 1:
        longit.append(1 * np.pi / 180)
    else:
        longit.append(0 * np.pi / 180)

    #================================================
    # 4. Starting the controller
    #================================================

    if t[i] >= 15:
        longitgrd = 0.2 * pitch[i] * 180 / np.pi  # P controller uses longit in deg
        longit[i] = longitgrd * np.pi / 180  # in rad

    c = u[i] * np.sin(pitch[i]) - w[i] * np.cos(pitch[i])
    h = -z[i]
    collect.append(collect[0])

    # Defining the differential equations

    #================================================
    # 5. See calculation of parameters above
    #================================================
    # Defining the nondimensional notations
    qdiml = q[i] / omega
    vdiml = np.sqrt(u[i]**2 + w[i]**2) / vtip
    phi = np.arctan2(w[i], u[i])
    alfc = longit[i] - phi

    mu = vdiml * np.cos(alfc)
    labc = vdiml * np.sin(alfc)

    # a1 Flapping calculation
    teller = -16 / lok * qdiml + 8 / 3 * mu * collect[i] - 2 * mu * (labc + labi[i])
    a1 = teller / (1 - 0.5 * mu**2)

    # the thrust coefficient
    ctelem = cla * volh / 4 * (2 / 3 * collect[i] * (1 + 1.5 * mu**2) - (labc + labi[i]))
    # Thrust coefficient from Glauert
    alfd = alfc - a1
    ctglau = 2 * labi[i] * np.sqrt((vdiml * np.cos(alfd))**2 + (vdiml * np.sin(alfd) + labi[i])**2)

    #================================================
    # 6. See equatios of motion below
    #================================================
    # Equations of motion
    labidot = ctelem
    thrust = labidot * rho * vtip**2 * area
    helling = longit[i] - a1
    vv = vdiml * vtip

    udot = -g * np.sin(pitch[i]) - cds / mass * 0.5 * rho * u[i] * vv + thrust / mass * np.sin(helling) - q[i] * w[i]
    wdot = g * np.cos(pitch[i]) - cds / mass * 0.5 * rho * w[i] * vv - thrust / mass * np.cos(helling) + q[i] * u[i]
    qdot = -thrust * mast / iy * np.sin(helling)
    pitchdot = q[i]
    xdot = u[i] * np.cos(pitch[i]) + w[i] * np.sin(pitch[i])
    zdot = -c

    labidot = (ctelem - ctglau) / tau

    u.append(u[i] + stap * udot)
    w.append(w[i] + stap * wdot)
    q.append(q[i] + stap * qdot)
    pitch.append(pitch[i] + stap * pitchdot)
    x.append(x[i] + stap * xdot)
    labi.append(labi[i] + stap * labidot)
    z.append(z[i] + stap * zdot)
    t.append(t[i] + stap)


## 6. Equation of Moiton


In order to start writing the equation of motion of the 3 degree of freedom helicopter model, first the forces acting on the helicopter need to be calculated, see figure. One can see that the following forces can be defined: thrust force $T$  , fuselage drag $D_{fus}$ and weight $W$  . These forces can be calculated in the program as:

$$
\begin{align}
T &= C_T \rho (\Omega R)^2 \pi R^2 \\
D_{fus} &= C_D \frac{1}{2} \rho V^2 S \\
W &= mg
\end{align}
$$

<br>
<p align="center"  id="figure1" style="background-color:white;">
<img src="https://github.com/MDPAV/HELICOR/blob/main/figures/eom_fig.png?raw=true" width="400" /> 
 <br>
<em>Figure 1: Free body diagram of a pitching helicopter</em>
</p>
<br>

The equations of  motion for the helicopter in the 3-dof model are given as:

$$
\begin{align}
\dot{u} &= -g \sin \theta_f - \frac{D_{fus}}{m} \cdot \frac{u}{V} + \frac{T}{m} \sin(\theta_c - a_1) - q w \\
\dot{w} &= g \cos \theta_f - \frac{D_{fus}}{m} \cdot \frac{w}{V} - \frac{T}{m} \cos(\theta_c - a_1) + q u \\
\dot{q} &= -\frac{T}{I_y} h \sin(\theta_c - a_1) \\
\dot{\theta} &= q
\end{align}
$$

To these equations, one more equation of motion was added in the simulation representing the rotor “quasi-dynamic” inflow. More precisely, treating induced velocity vi as a state variable results in adding a new equation on dvi/dt  to the helicopter system of equations of motion of form:

$$
\begin{equation}
\tau \frac{dv_i}{dt} = C_{T,\text{elem}} - C_{T,\text{Glauert}}
\end{equation}
$$

where $\tau$ is the rotor inflow time constant. This can be considered in the simulation between 0.1 and 0.5 sec. Introducing the quasi-inflow modeling represent not only is an efficient method to compute the rotor instantaneous induced velocity vi but it allows also to consider the time needed for the wake to achieve a new value during helicopter maneuvering flight.

## 7. Numerical Integration 

Once that we have defined the equations of motion, integrating each time step these equations result in representing the system state variables. Different integration methos can be chosen to solve numerically the differential equations, ex. Euler, Runge Kutta,  etc. (see also [wikipedia](https://en.wikipedia.org/wiki/Numerical_methods_for_ordinary_differential_equations#Methods)) . In this program we have chosen to use a simple Euler integration method.




## 7. Plotting results

We can now plot different variables representing the motion of the helicopter and the pilot controls during the time we consider the simulation. 

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=u, mode='lines', name='u(m/s)'))
fig.update_layout(title='Velocity (u) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='u(m/s)')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=np.array(pitch) * 180 / np.pi, mode='lines', name='pitch(deg)'))
fig.update_layout(title='Pitch over Time',
                  xaxis_title='t (s)',
                  yaxis_title='pitch(deg)')
fig.show()


In [ ]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=x, mode='lines', name='x(m)'))
fig.update_layout(title='Displacement in X-Direction over Time',
                  xaxis_title='t (s)',
                  yaxis_title='x(m)')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=w, mode='lines', name='w(m/s)'))
fig.update_layout(title='Vertical Velocity (w) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='w(m/s)')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=q, mode='lines', name='q(rad/s)'))
fig.update_layout(title='Rate of Pitch Change (q) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='q(rad/s)')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=labi, mode='lines', name='labi'))
fig.update_layout(title='Non-Dimensional Inflow (labi) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='labi')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=-np.array(z), mode='lines', name='h(m)'))
fig.update_layout(title='Altitude (h) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='h(m)')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=np.array(longit)*180/np.pi, mode='lines', name='longit(deg)'))
fig.update_layout(title='Longitudinal Input (longit) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='longit(deg)')
fig.show()
